In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import json
import pandas as pd
import numpy as np
import os
from string import ascii_letters

In [2]:
import sys
sys.path.insert(0, './src')
from pso import PSOFeatureSelection
from sa import SimulatedAnnealing
from utils import OptimizationAlgorithm
from metrics import Metrics, SammonError, KruskalStress, EditDistance
from tqdm import tqdm, trange
import numpy as np
import pandas as pd
import datetime

import argparse
import yaml
from main import run_analysis, get_data_i
from utils import SubsetterArgparser
from utils import load_metrics_file
from typing import List, Tuple

import torch
import pickle
import gc
import os

from icecream import install

install()

In [3]:
data_path = os.path.join("data", "function_metrics.csv")
data, indices_size_descending = load_metrics_file(data_path)
device = torch.device('cuda')

In [4]:
repo_index = 50

In [5]:
index = indices_size_descending[repo_index]
repo_data = get_data_i(data, index, 'function')
repo_data = repo_data.to(device)

In [6]:
repo_data.shape

torch.Size([5133, 69])

In [7]:
K = 20
subset_size = 5

# Generate K unique random column indices
torch.manual_seed(43)
column_indices = torch.randperm(repo_data.shape[1])[:K]

# Select the K columns from the tensor
subset_repo = repo_data[:, column_indices]

In [8]:
subset_repo.shape

torch.Size([5133, 20])

## Enumerate all subsets to find best

In [26]:
z = torch.zeros(5)
z[torch.tensor([1, 0, 1, 1, 1], dtype=bool)] = 1
z

tensor([1., 0., 1., 1., 1.])

In [16]:
import itertools
from metrics import SammonError

sammon = SammonError(subset_repo)

subset_to_loss = []
X = torch.rand((3, 20), device='cuda')

for subset in itertools.combinations(list(range(K)), subset_size):
    # Create a binary mask tensor for the current subset
    mask_tensor = torch.zeros(K, dtype=torch.bool, device='cuda')
    mask_tensor[[idx for idx in subset]] = 1
    # loss = sammon.compute(mask_tensor)
    mask = torch.zeros(20, device='cuda')
    ic(mask_tensor)
    mask[mask_tensor] = 1
    ic(mask)
    # mask deselected to 0
    X_masked = X * mask
    ic(X_masked)
    # loss = sammon.compute(list(subset))
    # subset_to_loss.append((list(subset), loss))
    break

# subset_to_loss.sort(key=lambda x: x[1].item())

ic| mask_tensor: tensor([ True,  True,  True,  True,  True, False, False, False, False, False,
                         False, False, False, False, False, False, False, False, False, False],
                        device='cuda:0')


ic| mask: tensor([1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                  0., 0.], device='cuda:0')
ic| X_masked: tensor([[0.1627, 0.4747, 0.8382, 0.1235, 0.8998, 0.0000, 0.0000, 0.0000, 0.0000,
                       0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                       0.0000, 0.0000],
                      [0.6162, 0.1513, 0.3778, 0.2811, 0.1094, 0.0000, 0.0000, 0.0000, 0.0000,
                       0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                       0.0000, 0.0000],
                      [0.0966, 0.0022, 0.6156, 0.3085, 0.2219, 0.0000, 0.0000, 0.0000, 0.0000,
                       0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                       0.0000, 0.0000]], device='cuda:0')


In [12]:
subset_to_loss[:20]

[([2, 4, 14, 16, 19], tensor(0.1244, device='cuda:0')),
 ([2, 4, 15, 16, 19], tensor(0.1304, device='cuda:0')),
 ([4, 10, 14, 16, 19], tensor(0.1317, device='cuda:0')),
 ([0, 4, 14, 16, 19], tensor(0.1320, device='cuda:0')),
 ([2, 4, 10, 16, 19], tensor(0.1343, device='cuda:0')),
 ([2, 10, 14, 16, 19], tensor(0.1346, device='cuda:0')),
 ([0, 2, 14, 16, 19], tensor(0.1346, device='cuda:0')),
 ([4, 6, 14, 16, 19], tensor(0.1349, device='cuda:0')),
 ([2, 9, 14, 16, 19], tensor(0.1365, device='cuda:0')),
 ([4, 9, 14, 16, 19], tensor(0.1374, device='cuda:0')),
 ([2, 6, 14, 16, 19], tensor(0.1374, device='cuda:0')),
 ([2, 10, 15, 16, 19], tensor(0.1395, device='cuda:0')),
 ([0, 2, 15, 16, 19], tensor(0.1401, device='cuda:0')),
 ([4, 10, 15, 16, 19], tensor(0.1402, device='cuda:0')),
 ([0, 2, 4, 16, 19], tensor(0.1406, device='cuda:0')),
 ([0, 4, 15, 16, 19], tensor(0.1413, device='cuda:0')),
 ([2, 14, 16, 17, 19], tensor(0.1416, device='cuda:0')),
 ([2, 9, 15, 16, 19], tensor(0.1425, device=

In [27]:
from src.main import run_analysis
pbar = trange(0, 1)
selected_metrics_arr, optimums_arr = run_analysis(
    subset_repo,
    pbar,
    metric_subset_size=subset_size,
    error_function=SammonError,
    method=PSOFeatureSelection,
    seed=42,
    max_iter=10,
    n_runs=3,
    num_particles=10,
)

  0%|                                                      | 0/1 [00:01<?, ?it/s, N_metrics: 5; Run: 3/3; Opt: 0.170179]

In [28]:
optimums_arr

[0.21147753298282623, 0.17017912864685059, 0.15015943348407745]

In [29]:
from src.main import run_analysis
pbar = trange(0, 1)
selected_metrics_arr, optimums_arr = run_analysis(
    subset_repo,
    pbar,
    metric_subset_size=subset_size,
    error_function=SammonError,
    method=PSOFeatureSelection,
    seed=42,
    max_iter=10,
    n_runs=3,
    num_particles=10,
    selection_function='topk-small',
)


  0%|                                                      | 0/1 [00:01<?, ?it/s, N_metrics: 5; Run: 3/3; Opt: 0.170179]

  0%|                                                      | 0/1 [00:01<?, ?it/s, N_metrics: 5; Run: 3/3; Opt: 0.142484]

In [30]:
optimums_arr

[0.1814861297607422, 0.14248375594615936, 0.15150271356105804]

In [31]:
from src.main import run_analysis
pbar = trange(0, 1)
selected_metrics_arr, optimums_arr = run_analysis(
    subset_repo,
    pbar,
    metric_subset_size=subset_size,
    error_function=SammonError,
    method=PSOFeatureSelection,
    seed=42,
    max_iter=10,
    n_runs=3,
    num_particles=10,
    selection_function='rand',
)

  0%|                                                      | 0/1 [00:01<?, ?it/s, N_metrics: 5; Run: 3/3; Opt: 0.186431]

In [32]:
optimums_arr

[0.1884726881980896, 0.1864311397075653, 0.190450519323349]

In [13]:
# [x[:subset_size] for x in selected_metrics_arr]

[[[0.6277675628662109,
   0.3613280951976776,
   0.7820790410041809,
   0.7051138877868652,
   0.3471134305000305,
   0.4170715808868408,
   1.1624526977539062,
   0.7161104679107666,
   0.9573154449462891,
   0.5850797891616821,
   0.7505536079406738,
   0.37281668186187744,
   0.4622475206851959,
   0.1166602373123169,
   0.4569224715232849],
  [0.7954239845275879,
   0.3523367941379547,
   1.3883907794952393,
   0.14424923062324524,
   0.02078908681869507,
   0.393947958946228,
   0.566846489906311,
   0.7683454751968384,
   0.35238927602767944,
   0.3195575475692749,
   0.6609660387039185,
   0.532672107219696,
   1.0118502378463745,
   0.7494090795516968,
   0.8064063787460327],
  [-0.03348433971405029,
   0.32701823115348816,
   0.9217746257781982,
   0.2141723930835724,
   0.08300352096557617,
   0.5461053848266602,
   0.6119529008865356,
   0.2175091952085495,
   0.14072826504707336,
   0.75847989320755,
   0.08367013931274414,
   0.46088171005249023,
   0.7224437594413757,
   